In [ ]:
# !pip install -r ../requirements.txt

In [1]:
import pandas as pd

base_path = "../data/"  

orders = pd.read_csv(base_path + "olist_orders_dataset.csv")
reviews = pd.read_csv(base_path + "olist_order_reviews_dataset.csv")
items = pd.read_csv(base_path + "olist_order_items_dataset.csv")
customers = pd.read_csv(base_path + "olist_customers_dataset.csv")

print("orders:", orders.shape)
print("reviews:", reviews.shape)
print("items:", items.shape)
print("customers:", customers.shape)

orders.head()


orders: (99441, 8)
reviews: (99224, 7)
items: (112650, 7)
customers: (99441, 5)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [2]:
# 주문 날짜를 datetime으로 변환
orders["order_purchase_timestamp"] = pd.to_datetime(orders["order_purchase_timestamp"])

# 고객별 총 주문 수와 마지막 주문 날짜 계산
customer_orders = orders.groupby("customer_id").agg(
    total_orders=("order_id", "nunique"),
    last_order_date=("order_purchase_timestamp", "max")
).reset_index()

# 확인
customer_orders.head()


,customer_id,total_orders,last_order_date
0,00012a2ce6f8dcda20d059ce98491703,1,2017-11-14 16:08:26
1,000161a058600d5901f007fab4c27140,1,2017-07-16 09:40:32
2,0001fd6190edaaf884bcaf3d49edf079,1,2017-02-28 11:06:43
3,0002414f95344307404f0ace7a26f1d5,1,2017-08-16 13:09:20
4,000379cdec625522490c315e70c7a9fb,1,2018-04-02 13:42:17


In [3]:
orders_reviews = pd.merge(orders, reviews, on="order_id", how="inner")


In [4]:
avg_review_scores = orders_reviews.groupby("customer_id").agg(
    avg_review_score=("review_score", "mean")
).reset_index()


In [5]:
customer_summary = pd.merge(customer_orders, avg_review_scores, on="customer_id", how="left")
customer_summary.head()


,customer_id,total_orders,last_order_date,avg_review_score
0,00012a2ce6f8dcda20d059ce98491703,1,2017-11-14 16:08:26,1.0
1,000161a058600d5901f007fab4c27140,1,2017-07-16 09:40:32,4.0
2,0001fd6190edaaf884bcaf3d49edf079,1,2017-02-28 11:06:43,5.0
3,0002414f95344307404f0ace7a26f1d5,1,2017-08-16 13:09:20,5.0
4,000379cdec625522490c315e70c7a9fb,1,2018-04-02 13:42:17,4.0


In [6]:
orders["order_delivered_customer_date"] = pd.to_datetime(orders["order_delivered_customer_date"])
orders["order_estimated_delivery_date"] = pd.to_datetime(orders["order_estimated_delivery_date"])


In [7]:
# 배송 지연일 = 실제 배송일 - 예상 배송일
orders["delivery_delay_days"] = (orders["order_delivered_customer_date"] - orders["order_estimated_delivery_date"]).dt.days


In [8]:
avg_delay = orders.groupby("customer_id").agg(
    avg_delivery_delay=("delivery_delay_days", "mean")
).reset_index()


In [9]:
customer_summary = pd.merge(customer_summary, avg_delay, on="customer_id", how="left")
customer_summary.head()


,customer_id,total_orders,last_order_date,avg_review_score,avg_delivery_delay
0,00012a2ce6f8dcda20d059ce98491703,1,2017-11-14 16:08:26,1.0,-6.0
1,000161a058600d5901f007fab4c27140,1,2017-07-16 09:40:32,4.0,-10.0
2,0001fd6190edaaf884bcaf3d49edf079,1,2017-02-28 11:06:43,5.0,-16.0
3,0002414f95344307404f0ace7a26f1d5,1,2017-08-16 13:09:20,5.0,-1.0
4,000379cdec625522490c315e70c7a9fb,1,2018-04-02 13:42:17,4.0,-5.0


In [10]:
# import sys
# !{sys.executable} -m pip install textblob


In [11]:
# import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')

In [12]:
# pip install deep-translator


In [13]:
from textblob import TextBlob
from deep_translator import GoogleTranslator
import pandas as pd

def get_sentiment_v2(text):
    if pd.isnull(text) or text.strip() == "":
        return None
    try:
        translated = GoogleTranslator(source='auto', target='en').translate(text)
        return TextBlob(translated).sentiment.polarity
    except:
        return None


In [16]:
from tqdm.auto import tqdm
tqdm.pandas()

sample_reviews = reviews.sample(100, random_state=42).copy()
sample_reviews["sentiment_score"] = sample_reviews["review_comment_message"].progress_apply(get_sentiment_v2)


  0%|          | 0/100 [00:00<?, ?it/s]

In [17]:
sample_reviews["sentiment_score"].describe()

count    39.000000
mean      0.376198
std       0.445199
min      -0.700000
25%       0.000000
50%       0.275000
75%       0.789167
max       1.000000
Name: sentiment_score, dtype: float64

In [18]:
sample_reviews[["review_comment_message", "sentiment_score"]].head(10)


,review_comment_message,sentiment_score
90252,NaN,NaN
24436,Cumpriu o acordado!,0.0
11313,NaN,NaN
75442,NaN,NaN
7217,NaN,NaN
72368,"Meu produto não foi entregue, sem me da nenhum...",0.0
63629,NaN,NaN
71390,material excelente alticima qulidade otimo,0.9
2873,Entregas separadas...ainda não chegou todo o p...,0.2
79100,NaN,NaN


In [ ]:
customer_summary["avg_sentiment_score"].describe()
